# The apply, map, and str.replace methods

In python it is common to loop through a database table and perform an operation on each record individually.

This is possible in GeoPandas as well. For instance we could use the loc or iloc method to loop through the rows and access each field value in that row and perform some operation on it.

In Pandas and GeoPandas, however, it is far more common to work on the entire DataFrame at once rather than row by row.  To do this we have to apply a function to an entire series using the apply method.

In [ ]:
%matplotlib inline
import geopandas as gpd

buowl = gpd.read_file("data/BUOWL_Habitat.shp")
raptor = gpd.read_file("data/Raptor_Nests.shp")
linear = gpd.read_file("data/Linear_Projects.shp")
eagles = gpd.read_file("data/BAEA_Nests.shp")

Let's say that we want to create a new field that contains the length(number of characters) of the recentstat field.  If we were new to Panda's we might try something like.

In [ ]:
raptor['stat_len'] = len(raptor['recentstat'])
raptor.head()

This clearly didn't work.  The length of a series is the number of rows in the DataFrame (879) not the number of characters for each row.  Our next step might be to loop through each row and assign the len of the recentstat field for every row. This is how we would typically do things in Python.

In [ ]:
import time
ts = time.time()
for idx in raptor.index:
    raptor.at[idx, 'stat_len'] = len(raptor.at[idx, 'recentstat'])
te = time.time()

method1 = te-ts
print("Total Time: {:10.5f}".format(method1))
raptor

This works but it is a brute force way and probably should never be done.  A better way is to use the apply method as follows

In [ ]:
ts = time.time()
raptor['stat_len']=raptor['recentstat'].apply(len)
te = time.time()
method2 = te-ts
print("Total Time: {:10.5f}".format(method2))
print(method1/method2)
raptor.tail()

Wow, notice that is almost 20x faster.  It may not make a big difference in this case with a simple function on a small database but if it was a more complex function on a larger database it could make a significant difference.  Think 1 minute vs 20 minutes or 1 hour vs. 20 hours.

Another option would be to apply a lambda function. Lambda's in python are simple unnamed functions that take an input and return a value in one line of code.

In [ ]:
ts = time.time()
raptor['stat_len']=raptor['recentstat'].apply(lambda x: int(len(x)/2))
te = time.time()
method3 = te-ts
print("Total Time: {:10.5f}".format(method3))
print(method1/method3)
raptor.tail()

This also works and is alot faster than the brute force method but not quite as fast as directly applying the len function

Still another method would be to write our own custom function.  This is overkill in this case as the len function already exists but as we'll see in a bit in other cases it may be our only option.  The advantage of this method in more complex cases is that we can include as many lines of code as we need.

In [ ]:
def return_len(s):
    return len(s)

ts = time.time()
raptor['stat_len']=raptor['recentstat'].apply(return_len)
te = time.time()
method3 = te-ts
print("Total Time: {:10.5f}".format(method3))
print(method1/method3)

Now lets think about the reason I interrupted this series on spatial function to discuss the apply method in the first place.  We want to create buffers for the raptor points, but the buffers will vary according to the species.  For Swainsons Hawks the buffer distance will be 333 meters and for everything else the buffer distance will be 667 meters.

First we need to convert the raptors GeoDataFrame to UTM coordinates so we can work with meters as the units.

In [ ]:
raptor.to_crs(epsg=26913, inplace=True)
raptor.head()

Next we want to create a new column called buf_dist that will hold the distance we will buffer the point by.  Since there isn't a pre-existing function to do this we will have to write our own custom function. We could use a Lambda function with a ternary operator that allows us to write a simple if-then statement in a single line.

In [ ]:
ts = time.time()
raptor['buf_dist'] = raptor['recentspec'].apply(lambda s: 333 if s=='Swainsons Hawk' else 667)
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor

That works but I think it is a bit confusing and also it is limited to a single condition.  We can write our own custom function and apply it and we can use it to evaluate as many conditions as we want.

In [ ]:
def calc_raptor_buffer(spec):
    if spec=='Swainsons Hawk':
        return 333
    elif spec=='Northern Harrier':
        return 500
    else:
        return 667
    
ts = time.time()
raptor['buf_dist'] = raptor['recentspec'].apply(calc_raptor_buffer)
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor

This also works, it is more flexible. its easier to understand, and it runs just as fast so its a win, win, win.

There is another method that can be used in these situations where we are "coding" a new column based on values in another column. The map function takes a dictionary as a parameter and returns the value when a key match is found.

First lets create the dictionary

In [ ]:
species_buffer = {"Swainsons Hawk":333, "Red-tail Hawk":667, "Northern Harrier":500}

Now lets use it to map a buffer distance value to the proper species

In [ ]:
ts = time.time()
raptor['buf_dist2']=raptor['recentspec'].map(species_buffer)
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor

Notice however that the last two values are NaN because they didn't match any key in the dictionary.  We can set a default value using a default dictionary.  These are a special type of dictionary that include a lambda function as the first element in the dictionary that returns a value when no key match is made.

To create a defaultdictionary we have to import the defaultdict class from the collections package.  Our lambda function will take no parameter because it will always just return the same value. In this case 1000

In [ ]:
from collections import defaultdict

species_buffer = defaultdict(lambda: 1000, {'Swainsons Hawk': 333, 'Red-tail Hawk': 667, 'Northern Harrier': 500})

In [ ]:
ts = time.time()
raptor['buf_dist']=raptor['recentspec'].map(species_buffer)
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor

Now notice that the SWHA and RTHA have a buffer distance of 1000.

Really though, we don't want SWHA and RTHA in the recentspec column. They were just entered by mistake by lazy people who didn't want to type out the full name and should be fixed.  First lets make sure that these are the only unwanted values using the unique method.

In [ ]:
raptor['recentspec'].unique()

It looks like that is the case.  We can use the str.replace method to change them to their proper values

In [ ]:
ts = time.time()
raptor['recentspec'].str.replace('SWHA', 'Swainsons Hawk')
raptor['recentspec'].str.replace('RTHA', 'Red-tail Hawk')
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor.tail()

Great except that our values haven't changed.  Well remember that in pandas most things are not really permanent unless you set the inplace parameter to true.  Except that replace has no inplace parameter.

But we can still make it permanent by assigning the result of the replace method to the original column.

In [ ]:
ts = time.time()
raptor['recentspec'] = raptor['recentspec'].str.replace('SWHA', 'Swainsons Hawk')
raptor['recentspec'] = raptor['recentspec'].str.replace('RTHA', 'Red-tail Hawk')
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor.tail()

In [ ]:
ts = time.time()
raptor['buf_dist']=raptor['recentspec'].map(species_buffer)
te = time.time()
print("Total Time: {:10.5f}".format(method3))
raptor

Although the pandas apply method is very fast, there are further performance gains that can be achieved. If you are working with large datasets and need improved performance you can look into vectorizing pandas functions. This blog post on [optinmizing pandas code for speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6) provides a very good place to start.